In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
import requests
from bs4 import BeautifulSoup

In [ ]:
data = pd.read_csv("moviename.csv")

In [ ]:
data.head(20)

,Title,Link
0,Bhaktha Chetha (1940 film),/wiki/Bhaktha_Chetha_(1940_film)
1,G. Ramanathan,/wiki/G._Ramanathan
2,Chandraguptha Chanakya,/wiki/Chandraguptha_Chanakya
3,C. N. Pandurangan,/wiki/C._N._Pandurangan
4,B. R. Panthulu,/wiki/B._R._Panthulu
5,T. R. Sundaram,/wiki/T._R._Sundaram
6,Jayakodi,/wiki/Jayakodi
7,Kalamegam,/wiki/Kalamegam
8,Krishnan Thoothu,/wiki/Krishnan_Thoothu
9,Papanasam Sivan,/wiki/Papanasam_Sivan


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7071 entries, 0 to 7070
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   7071 non-null   object
 1   Link    7071 non-null   object
dtypes: object(2)
memory usage: 110.6+ KB


In [ ]:
data = data.drop_duplicates()

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6721 entries, 0 to 7070
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   6721 non-null   object
 1   Link    6721 non-null   object
dtypes: object(2)
memory usage: 157.5+ KB


In [ ]:
n = 6680
tdt = data[n:n+1].values
tdt

array([['Ayothi', '/wiki/Ayothi']], dtype=object)

In [ ]:
tdt[0][1]

'/wiki/Ayothi'

In [ ]:
lnd = len(data)
lnd

6721

In [ ]:
movie_data = [["Title","wiki_url","img_url","Plot","Starring","Director","Writer","cinematographer","music","country","language"]]

In [ ]:
error_data = []
plot_balance = []

In [ ]:
for i in tqdm(range(lnd)):
  try:
    tdt = data[i:i+1].values
    url = 'https://en.wikipedia.org'+tdt[0][1]
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "html.parser")
    if response == 404:
      print(tdt[0][0])
      pass
    try:
      movbox = soup.find("table", {"class": "infobox vevent"})
      if movbox is not None:
        l = movbox.find("a",{'class':'image'})
        st = []
        u = l.find("img")['src']
        image = "https:"+u+" "
        dt = movbox.find_all("tr")
        lit = ["Directed by","Written by","Cinematography","Music by","Country","Language","Starring","Release date"]
        for op in dt:
          il = op.find("th",{'class':'infobox-label'})
          id = op.find("td",{'class':'infobox-data'})
          if id is not None and il is not None:
            if il.text in lit:
              if il.text == "Starring":
                uv = id.find_all("a")
                for m in uv:
                  st.append(m.text)
              elif il.text == "Release date":
                try:
                  date = id.find("li").text
                except:
                  date = "Date Not Found"
              elif il.text == "Directed by":
                director = id.text
              elif il.text == "Written by":
                writer = id.text
              elif il.text == "Cinematography":
                cinematographer = id.text
              elif il.text == "Music by":
                music = id.text
              elif il.text == "Country":
                country = id.text
              elif il.text == "Language":
                language = id.text
              else:
                print(il.text) 
        try:
          plot_section = soup.find('span',{'id':'Plot'})
          plot_text = ''
          for element in plot_section.findAllNext():
            if element.name == 'h2':
              break
            if element.name == 'p':
              plot_text += element.text
          movie_data.append([tdt[0][0],url,image,plot_text,st,director,writer,cinematographer,music,country,language])
        except:
          plot_balance.append(tdt[0][0])
    except:
      error_data.append([tdt[0][0],tdt[0][1]])    
  except:
    print("request error ",tdt[0][0],tdt[0][1])
      
  

  0%|          | 0/6721 [00:00<?, ?it/s]

request error  ta:போஜன் (திரைப்படம்) https://ta.wikipedia.org/wiki/%E0%AE%AA%E0%AF%8B%E0%AE%9C%E0%AE%A9%E0%AF%8D_(%E0%AE%A4%E0%AE%BF%E0%AE%B0%E0%AF%88%E0%AE%AA%E0%AF%8D%E0%AE%AA%E0%AE%9F%E0%AE%AE%E0%AF%8D)
request error  ta:பிரம்மரிஷி விஷ்வாமித்திரா https://ta.wikipedia.org/wiki/%E0%AE%AA%E0%AE%BF%E0%AE%B0%E0%AE%AE%E0%AF%8D%E0%AE%AE%E0%AE%B0%E0%AE%BF%E0%AE%B7%E0%AE%BF_%E0%AE%B5%E0%AE%BF%E0%AE%B7%E0%AF%8D%E0%AE%B5%E0%AE%BE%E0%AE%AE%E0%AE%BF%E0%AE%A4%E0%AF%8D%E0%AE%A4%E0%AE%BF%E0%AE%B0%E0%AE%BE
request error  ta:புரட்சி வீரன் https://ta.wikipedia.org/wiki/%E0%AE%AA%E0%AF%81%E0%AE%B0%E0%AE%9F%E0%AF%8D%E0%AE%9A%E0%AE%BF_%E0%AE%B5%E0%AF%80%E0%AE%B0%E0%AE%A9%E0%AF%8D
request error  ta:மனோரதம் https://ta.wikipedia.org/wiki/%E0%AE%AE%E0%AE%A9%E0%AF%8B%E0%AE%B0%E0%AE%A4%E0%AE%AE%E0%AF%8D
request error  ta:மகா பாரதம் (திரைப்படம்) https://ta.wikipedia.org/wiki/%E0%AE%AE%E0%AE%95%E0%AE%BE_%E0%AE%AA%E0%AE%BE%E0%AE%B0%E0%AE%A4%E0%AE%AE%E0%AF%8D_(%E0%AE%A4%E0%AE%BF%E0%AE%B0%E0%AF%88%E0%AE%AA%E0%AF%8D

In [ ]:
len(error_data),len(plot_balance)

(1094, 507)

In [ ]:
import csv
with open("plotndata.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for row in plot_balance:
        writer.writerow(row)

In [ ]:
print(len(movie_data))

3588


In [ ]:
import csv
with open("moviedata.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    for row in movie_data:
        writer.writerow(row)

In [ ]:
datacheck = pd.read_csv("moviedata.csv")

In [ ]:
datacheck.head(5)

,Title,wiki_url,img_url,Plot,Starring,Director,Writer,cinematographer,music,country,language
0,Bhaktha Chetha (1940 film),https://en.wikipedia.org/wiki/Bhaktha_Chetha_(...,https://upload.wikimedia.org/wikipedia/en/thum...,Chetha (Papanasam Sivan) is a cobbler of a low...,"['Papanasam Sivan', 'Kothamangalam Subbu']",K. Subrahmanyam,K. Subrahmanyam (dialogues),Kamal Ghosh,V. S. Parthasarathy Iyengar[1],India,Tamil
1,Jayakodi,https://en.wikipedia.org/wiki/Jayakodi,https://upload.wikimedia.org/wikipedia/en/f/fe...,"Rajam, an impoverished Brahmin girl, is unmarr...",[],Bhagwan Dada,K. Subrahmanyam (dialogues),Ahmedulla,C. Ramchandra,India,Tamil
2,Kalamegam,https://en.wikipedia.org/wiki/Kalamegam,https://upload.wikimedia.org/wikipedia/en/e/e1...,Kalamegam is based on the life of 15th century...,[],Ellis R. DunganWilliam J. Moylan,Puduvai Bharathidasan,V. Namasivayam,R. N. Chinnaiah,India,Tamil
3,Krishnan Thoothu,https://en.wikipedia.org/wiki/Krishnan_Thoothu,https://upload.wikimedia.org/wikipedia/commons...,The story of the film was of the history of Lo...,[],R. Prakash,Puduvai Bharathidasan,Kamal Ghosh,R. N. Chinnaiah,India,Tamil
4,Naveena Vikramadityan,https://en.wikipedia.org/wiki/Naveena_Vikramad...,https://upload.wikimedia.org/wikipedia/commons...,"In the film, N. S. Krishnan used to give music...","['N. S. Krishnan', 'T. A. Mathuram', 'T. S. Du...",K. S. Mani,Puduvai Bharathidasan,Kamal Ghosh,N. S. Balakrishnan,India,Tamil


In [ ]:
"""url = 'https://en.wikipedia.org'+tdt[0][1]
print(url)"""

https://en.wikipedia.org/wiki/Ayothi


In [ ]:
response = requests.get(url)
html = response.content
soup = BeautifulSoup(html, "html.parser")

In [ ]:
response

<Response [200]>

In [ ]:
#soup

In [ ]:
movbox = soup.find("table", {"class": "infobox vevent"})

In [ ]:
if movbox is not None:
  l = movbox.find("a",{'class':'image'})
  u = l.find("img")['src']
  print("https:"+u)

https://upload.wikimedia.org/wikipedia/en/thumb/c/c5/Ayothi_Poster.jpeg/220px-Ayothi_Poster.jpeg


In [ ]:
dt = movbox.find_all("tr")
lit = ["Directed by","Written by","Cinematography","Music by","Country","Language","Starring","Release date"]
for i in dt:
  il = i.find("th",{'class':'infobox-label'})
  id = i.find("td",{'class':'infobox-data'})
  if id is not None and il is not None:
    if il.text in lit:
      if il.text == "Starring":
        print("Starring ",end="||| ")
        uv = id.find_all("a")
        for m in uv:
          print(m.text,end=" , ")
        print()
      elif il.text == "Release date":
        try:
          print("Release Date ",end="||| ")
          print(id.find("li").text)
        except:
          print("Date Not Found")
      else:
        print(il.text,"|||",id.text)

Directed by ||| R. Manthira Moorthy
Written by ||| R. Manthira Moorthy
Starring ||| M. Sasikumar , Preethi Asrani , Yashpal Sharma , Pugazh , 
Cinematography ||| Madhesh Manickam
Music by ||| N. R. Raghunanthan
Release Date ||| 3 March 2023 (2023-03-03)
Country ||| India
Language ||| Tamil


In [ ]:
plot_section = soup.find('span',{'id':'Plot'})
plot_text = ''
for element in plot_section.findAllNext():
    if element.name == 'h2':
        break
    if element.name == 'p':
        plot_text += element.text

In [ ]:
print(plot_text)

Balram (Yashpal Sharma) a conservative Ayodhya Hindu plans a pilgrimage to Rameshwaram along with his wife Janaki, daughter Shivani (Preethi Asrani), and son Sonu. On the way to Rameshwaram, Balram picks up a fight with the cab driver and this leads to an accident.
Janaki is heavily injured and admitted to the ICU, and immediately requires to be shifted to another hospital to ensure survival. Balram and his children are helpless as they can't understand the language of the locals. A friend of the Cab Driver (M. Sasikumar) witnesses the helpless children who had just lost their mother and decides to help. He drives the ambulance to Madurai Government Hospital, but Janaki passes away en route. Balram opposes post-mortem on his wife, citing religious faith, and the ambulance drives to the airport. The ambulance driver is stopped by the airport authorities and the police, while the latter takes them to police station for inquiry on the death of Janaki. They are proven innocent and are take